In [ ]:
#加载包
import numpy as np
import pandas as pd 
import os
import pathlib
import PIL
from pathlib import Path
from PIL import Image, ImageDraw
from math import sqrt
import ast

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv")
test = pd.read_csv("../input/tensorflow-great-barrier-reef/test.csv")
sub = pd.read_csv("../input/tensorflow-great-barrier-reef/example_sample_submission.csv")

path = Path('../input/tensorflow-great-barrier-reef/train_images')
filepaths = list(path.glob(r'**/*.jpg'))

In [ ]:
#查看数据的长度
print("Number of training samples: ", len(train))
print("Number of testing samples: ", len(test))

In [ ]:
train.head(150)

In [ ]:
train_clean = train.loc[train["annotations"] != "[]"]
print(f"No starfishes in {len(train)-len(train_clean)} samples.")
print(f"The clean train set has {len(train_clean)} images for us to work with.")

In [ ]:
train_clean.head()

In [ ]:
#查看序列数
len(train_clean.sequence.value_counts())

In [ ]:
#每个序列的行数
print("Sequence Samples")
print(train_clean.sequence.value_counts())

In [ ]:
seq_df = train_clean.sequence.value_counts().to_frame()
plt.figure(figsize=(16, 9))
sns.barplot(x=seq_df.index, y=list(seq_df.sequence), palette="Greens_d")
plt.title("Distribution of Sequences")
plt.xlabel("Sequence Id")
plt.ylabel("Frequency")
plt.show()

In [ ]:
num_boxes = []
annotations_clean = []
for elem in train_clean.annotations:
    ann = ast.literal_eval(elem)
    num_boxes.append(len(ann))
    annotations_clean.append(ann)

In [ ]:
#每行添加num框，并将annotations列更改为适当的python可解析字典列表
train_clean["num_boxes"] = num_boxes
train_clean["annotations"] = annotations_clean

In [ ]:
train_clean.head()

In [ ]:
print("#box Frequency")
print(train_clean.num_boxes.value_counts())

In [ ]:
#清洁训练集中的边界框数量
print(f"Number of Bounding Boxes in the dataset: {train_clean.num_boxes.sum()}")

In [ ]:
box_count = train_clean.num_boxes.value_counts().to_frame()

In [ ]:
plt.figure(figsize=(16, 9))
sns.barplot(x=box_count.index, y=list(box_count.num_boxes), palette="Greens_d")
plt.title("Distribution of Num_boxes")
plt.xlabel("# of Boxes")
plt.ylabel("Frequency")
plt.show()

In [ ]:
list(train_clean["annotations"])[0]

In [ ]:
src = '../input/tensorflow-great-barrier-reef/train_images'
paths = []
for row in train_clean.image_id:
    vid_num = row.split('-')[0]
    img_num = row.split('-')[1]
    paths.append(os.path.join(src,f'video_{vid_num}',img_num+'.jpg'))

In [ ]:
train_clean['paths'] = paths

In [ ]:
def vis_boxes(img_path, bboxes):
    coords = []
    for box in bboxes:
        x1 = box['x']
        y1 = box['y']
        x2 = x1 + box['width']
        y2 = y1 + box['height']
        coords.append([x1, y1, x2, y2])
        
    img = Image.open(img_path)
    img1 = img.copy()
    draw = ImageDraw.Draw(img1)
    for elem in coords:
        draw.rectangle(elem, outline='red', width=7)
    
    return img1

In [ ]:
train_clean.head()

In [ ]:
train_clean.groupby('sequence').num_boxes.sum().to_frame()

In [ ]:
plt.figure(figsize=(16, 9))
n_images = 9
count = 0
r,c = int(sqrt(n_images)), int(sqrt(n_images))
train_plot = train_clean.sample(n = n_images)

for _, row in train_plot.iterrows():
    img_path = row['paths']
    bboxes = row['annotations']
    plt.subplot(r, c, count + 1)
    img_out = vis_boxes(img_path, bboxes)
    plt.imshow(img_out)
    count+=1

plt.show()
plt.tight_layout()